# Benchmark ReQA
- Input: `06_calculate_distance.parquet`

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from utils import encode_in_batch

import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
data = pd.read_parquet('data/06_calculate_distance.parquet')
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_llm_paraphrase_aug,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.654346,0.333436,0.942305,0.901118,0.901118,0.783473,0.677738,0.642690,0.595411,0.224245
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.600163,0.285477,0.712166,0.668647,0.792416,0.537381,0.611034,0.867701,0.387706,0.000000
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.690551,0.227241,0.896181,0.896181,0.617513,0.617513,0.750872,0.750872,0.392775,0.000000
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.659828,0.469348,0.723428,0.849254,0.870112,0.849254,0.528563,0.648129,0.376103,0.276969
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.630858,0.627320,0.716229,0.672312,0.727237,0.751295,0.794606,0.794606,0.574593,0.631798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.911251,0.195957,0.880506,0.930000,0.899868,0.930000,0.910610,0.905050,0.312825,0.522812
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.880787,0.476029,0.900869,0.820951,0.922494,0.922002,0.901534,0.925271,0.458451,0.00000

In [4]:
# Add None so that the first benchmark only consists of the original questions
all_augment_cols = [None] + [col for col in data.columns if col.startswith('th_')]
all_augment_cols

['th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_wordnet_aug']

In [5]:
model_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-06-06 10:22:55.040296: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


: 

: 

In [3]:
def evaluate(question_id,question_all,context_id,context_all,mrr_rank=10,status=True):
    top_1 = 0; top_5 = 0; top_10 = 0;
    mrr_score = 0
    context_id = np.array(context_id)
    sim_score = np.inner(question_all,context_all)
    if status == True:
        status_bar = enumerate(tqdm(sim_score))
    else:
        status_bar = enumerate(sim_score)
    for idx,sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        try:
            idx_search = list(index_edit).index(question_id[idx])
        except:
            idx_search = 999999
        if idx_search == 0:
            top_1+=1
            top_5+=1
            top_10+=1
        elif idx_search < 5:
            top_5+=1
            top_10+=1
        elif idx_search < 10:
            top_10+=1  
        if idx_search < mrr_rank:
            mrr_score += (1/(idx_search+1))
    mrr_score/=len(question_all)
    return top_1,top_5,top_10,mrr_score

In [ ]:
def benchmark_single(data, question_col="questions", aug_col=None, model_embed=model_embed, filter_zero_dist=True):
    # Check if column has a distance counterpart
    questions = data[question_col].values

    question_all = encode_in_batch(model_embed, questions)
    context_all = encode_in_batch(model_embed, data["context"].values)

    question_ids = data["id"].values
    context_ids = data["id"].values

    if aug_col:
        augments = data[aug_col].values

        if filter_zero_dist:
            distances = data[aug_col.replace("th_", "dis_")].values
            augments = augments[distances > 0]
            augment_ids = data["id"].values[distances > 0]
        else:
            augment_ids = data["id"].values
        
        augment_all = encode_in_batch(model_embed, augments)
        
        # Augment into context
        context_all = np.concatenate([context_all, augment_all], axis=0)
        context_ids = np.concatenate([context_ids, augment_ids], axis=0)

    top_1, top_5, top_10, mrr_score = evaluate(
        question_ids, question_all, context_ids, context_all, status=True
    )
    return top_1, top_5, top_10, mrr_score

In [ ]:
results = []

for col in tqdm(all_augment_cols):
    top_1, top_5, top_10, mrr_score = benchmark_single(data, aug_col=col)
    print(f"{col}: top_1: {top_1}, top_5: {top_5}, top_10: {top_10}, mrr_score: {mrr_score}")
    results.append({"name": col, "top_1": top_1, "top_5": top_5, "top_10": top_10, "mrr_score": mrr_score})

results = pd.DataFrame(results)
results

In [ ]:
results.to_csv("results/07_benchmark_reqa.csv", index=False)